In [ ]:
#so what do we want
#we want to utilize batching of neural network inputs (different from normal MCTS, so we cant just blindly follow
#the example of the other paper)
#and we also would like to maximize CPU and GPU resource utilization

#so the pipeline thing does seem to be one of the best solutions for maximizing resource utilization
#but for batching, I'm not totally sure.

#so what would be the flow? we should write some pseudo code
#for _ in number of sims -> send curr node?
#otherwise we would always set the curr node to be a copy of the root node
#thats probably okay

#so basically we send some _ input into select, it doesnt matter, 
# so for task in range(num_sims), None
#pipe.put(task)

#then we need to specify the linking
#basically what we want:
#select(_) -> evaluate() -> expand(policy, value) -> backup(value)
#so that's pretty simple, but the tricky part will be how to utilize batching with this
#what we would need to do would be somehow to accumulate in evaluate, and then send it off to a simple queue
#or something, and wait for the policy and value to return, so we would have normal multi processing queueing
#inside of the pipe

#that doesn't sound impossible by any means

In [1]:
import numpy as np
from ipdb import set_trace
from copy import deepcopy as dc

class AlphaZero:
    def __init__(self, controller, turns_until_tau0=15, alpha=None, epsilon=.25, c=1):
        self.turns_until_tau0 = turns_until_tau0
        if alpha is None:
            alpha = min(10/len(policy), 1)
        else:
            self.alpha = alpha
        
        self.epsilon = epsilon
        self.c = c
        
        self.max_depth = controller.max_depth

        self.root_node = {
            "children": None,
            "parent": None,
            "N": 0,
            "d": 0
        }
        
        self.controller = controller
        self.starting_indices = starting_indices
        self.decision_list

        self.turn = 1

        self.T = 1

    @staticmethod
    def select(_):
        node = self.root_node
        
        trajectory = []
        while node["children"] is not None and node["d"] < self.max_depth:
            choice_idx = node["max_uct_idx"]
            node["max_uct_idx"] ["children"][choice_idx]
            node = node["children"][choice_idx]
            node["max_uct"]

            d = node["d"]
            decision_idx = d % len(decision_list)
            starting_idx = starting_indices[decision_idx]
            emb_idx = starting_idx + choice_idx

            trajectory.append(emb_idx)

        return trajectory

    def select_real(self, node, stochastic=True):
        visits = np.array([child["N"] for child in node["children"]])

        if self.T != 0:
            visits_sum = (1.0 * visits.sum())

            if visits_sum == 0:
                set_trace()

            assert visits_sum != 0
            
            visits = visits / visits_sum
            idx = np.random.choice(len(visits), p=visits)
        else:
            idx = np.argmax(visits)

        # if self.turn == self.turns_until_tau0:
        #     self.T = 0

        self.turn += 1

        node = node["children"][idx]
        node["parent"] = None

        return idx, visits, node

    @staticmethod
    def get_policy_and_value(trajectory):
        indices = []
        weights = dict()
        seen = dict()

        scaled_one = 1/len(trajectory)
        for emb_idx in trajectory:
            try:
                seen[emb_idx]
            except:
                seen[emb_idx] = True

                try:
                    weights[emb_idx] += scaled_one
                except:
                    weights[emb_idx] = scaled_one

                indices.append(emb_idx)
        
        logits = []
        embeddings = []
        weights_list = []
        for _, idx in enumerate(indices):
            embeddings.append(self.embeddings[idx].unsqueeze(0))
            logits.append(self.emb_merge_pre_softmax(self.embeddings[idx])[indices].unsqueeze(0))
            weights_list.append(weights[idx])
        weights = torch.from_numpy(np.array(weights_list)).float().unsqueeze(-1)
        weights = weights.view(1, -1)
        logits = torch.cat(logits)

        probas = F.softmax(logits)
        probas = weights.mm(probas)

        probas2 = F.softmax(logits)*weights
        probas2 = probas2.sum(0)

        assert (probas == probas2).all()

        probas /= probas.sum() #normalize
        probas = probas.view(1, -1)
        embeddings = torch.cat(embeddings)
        final_emb = probas.mm(embeddings)

        final_emb = final_emb.view(1, 1, -1)

        cont_out = self.controller(final_emb)
        
        logits = self.softmaxs[decision_idx](cont_out).squeeze()
        probas = F.softmax(logits)

        return cont_out
    
    @staticmethod
    def evaluate(node, trajectory):
        depth = node["d"]
        layer_idx = depth // len(self.decision_list)
        decision_idx = depth % len(self.decision_list)
        decision_name = self.decision_list[decision_idx]

        while True:
            skip_curr = self.check_condition(az, layer_idx, decision_name)
            if not skip_curr:
                break
            else:
                az.curr_node["d"] += 1
                depth = az.curr_node["d"]
                layer_idx = depth // len(self.decision_list)
                decision_idx = depth % len(self.decision_list)
                decision_name = self.decision_list[decision_idx]

        if len(trajectory) > 0:
            cont_out = get_policy_and_value(trajectory)

        logits = self.softmaxs[decision_idx](cont_out).squeeze()
        probas = F.softmax(logits)

        probas_np = probas.detach().data.numpy()
        # for name, arr in self.decisions.items():
        #     if decision_name is name:
        #         assert len(probas_np) == len(arr)

        if self.mask_conditions[decision_name] is not None:
            probas_np = self.mask_conditions[decision_name](layer_idx, probas_np)


    #we should try to calculate as muchas possible outside this class
    #so lets try to only pass what we need
    def expand(self, policy):
        self.curr_node["children"] = []

        if self.curr_node["parent"] is None:
            policy = self.add_dirichlet_noise(policy)

        for p in policy:
            child = {
                "N": 0,
                "W": 0,
                "Q": 0,
                "U": p,
                "P": p,
                "d": self.curr_node["d"]+1,
                "children": None,
                "parent": self.curr_node
            }

            self.curr_node["children"].extend([child])

        return self.curr_node

    def update_uct(self, node):
        node["children_ucts"] = []
        for i, child in enumerate(node["children"]):
            child["U"] = self.c*child["P"] * \
                (1 + np.log(node["N"])/(1 + child["N"]))
            child["UCT"] = child["Q"] + child["U"]
            node["children_ucts"].append(child["UCT"])
            
        node["children_ucts"] = np.array(node["children_ucts"])
        
        return node

    @staticmethod
    def backup(node, value):
        value += 1
        value /= 2
        while node["parent"] is not None:
            node = self.update_node(node, value)

            node = self.update_uct(node)
            
            node = node["parent"]

        node["N"] += 1

        node = self.update_uct(node)
        
        return node

    def update_node(self, node, value):
        node["N"] += 1
        node["W"] += value
        node["Q"] = node["W"]/node["N"]
        
        return node

    def add_dirichlet_noise(self, policy):
        alpha = self.alpha 
        epsilon = self.epsilon
        epsilon = 0.25
        nu = np.random.dirichlet([alpha] * len(policy))
        policy = policy*(1-epsilon) + nu*epsilon

        return policy

In [2]:
from mpipe import OrderedStage, Pipeline

In [4]:
az = AlphaZero(max_depth=10)

In [ ]:
select = az.select
backup = az.backup
up

In [ ]:
# s1 = OrderedStage(select)
# s2 = OrderedStage(evaluate)
# p = Pipeline(s1.link(s2))

# for task in 1, 2, 3, 4, 5, None:
#     p.put(task)